<a href="https://colab.research.google.com/github/dbosnacki/HelisDeepLearningCourse/blob/main/exercises/ExerciseModelTrainTestProteinDomainsRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Import
Import various libraries and funcitons which will be used in the code

In [ ]:
import time
import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Data import

*   We use version 2.4 of the CATH database of protein structures https://www.cathdb.info/

*  A preprocessed copy of the database from 'https://raw.githubusercontent.com/dbosnacki/HelisDeepLearningCourse/main/cath-domain-description-file-v2_4ProcessedForNN.tsv' 
as a tab separated text document
*   You can import this kind of data files (CSV) with pandas or numpy






In [ ]:
#Loading, padding and one-hot encoding of the data
url = 'https://raw.githubusercontent.com/dbosnacki/HelisDeepLearningCourse/main/cath-domain-description-file-v2_4ProcessedForNN.tsv' 
df = pd.read_csv(url, delimiter = "\t", header=None)

# Data preprocessing


*   Extract from the data frame the second and third column which correspond to the labes and the sequences ofo the protein domains, respectively
*   Pad the sequences up to some maximal length
*   Implement one-hot encoding of the sequences






In [ ]:
# generate the signature (features) of the sequences based on pair frequencies
npd = df[3].to_numpy()

# extract the sequences list from the data frame
dataset = []          
        
maxSeqLength = 0
for sequence in npd:
    l = len(sequence)
    if l > maxSeqLength:
      maxSeqLength = l 

data = list()
for i in range(len(npd)):
  alph_as_num = np.array(list(npd[i])).view(np.int32)
  data.append(np.array(list(npd[i])).view(np.int32))

padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    data, padding="post"
)
#print(padded_inputs)

#labels = pd.DataFrame(df).to_numpy()
labels = list(df[2])   

# Building, training and evaulating the model


*   Assign the data and labels to the corresponding variables
*   Use cross validation to evaluate the results
*   Build a deep neural network model to predict the protein domain class
*   Train and test/evaluate the model at the end of each fold using some metric, e.g., accuracy
*   Save the evaluation results for each folt in a list












## Assign the data and the labels and split them into training and test/ealuation sets

In [ ]:
X = padded_inputs
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Building the model


*   Compose the model layer by layer
*   Print a summary
*   Compile the model





In [ ]:
# Define RNN model 
model = keras.Sequential(
        [
           layers.Embedding(128, 64),
           layers.Bidirectional(layers.LSTM(128, dropout = 0.4, name="rnn1")),
           #layers.Dense(64, activation="relu", name="layer2"),
           layers.Dense(3, activation="softmax", name="output"),
        ]
    )
    
model.summary()

model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'],
        optimizer='adam',
    )

## Training
Train the model using the training part of the data

In [ ]:
history = model.fit(X_train[train_index], 
                        y_train[train_index], 
                        batch_size = 128, 
                        epochs = 100, 
                        #class_weight = class_weight, 
                        validation_data = (X_train[val_index], y_train[val_index]),
                        #callbacks = callbacks_list,
                        verbose = 2)

## Evaluation
Evaluta the model using the test part of the data

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=2)
#model.save(newpath + r'\fold-' + str(fold_no) + '.hdf5') 
#print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

## Cross-validation
* Put together the building, training and evalution steps in a cross-validation loop
* collect the mettric, e.g., accuracy, as well as the loss for each fold in a list or dictionary such that it can be processed later
X = padded_inputs
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X = padded_inputs
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

accuracy_per_fold = []
loss_per_fold = []

fold_no = 1
seed = 10
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for train_index, val_index in skf.split(X_train, y_train):
    
    print('Fold: ' + str(fold_no))
    

    # Define RNN model 
    model = keras.Sequential(
        [
           layers.Embedding(128, 64),
           layers.Bidirectional(layers.LSTM(128, dropout = 0.4, name="rnn1")),
           #layers.Dense(64, activation="relu", name="layer2"),
           layers.Dense(3, activation="softmax", name="output"),
        ]
    )
    
    # model.summary()

    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'],
        optimizer='adam',
    )
    

    history = model.fit(X_train[train_index], 
                        y_train[train_index], 
                        batch_size = 128, 
                        epochs = 100, 
                        #class_weight = class_weight, 
                        validation_data = (X_train[val_index], y_train[val_index]),
                        #callbacks = callbacks_list,
                        verbose = 2)
    
    scores = model.evaluate(X_test, y_test, verbose=2)
    
    #model.save(newpath + r'\fold-' + str(fold_no) + '.hdf5') 

    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    accuracy_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
   

# Evaluation results
* Print the saved evalutaion results for each of the fold as well as some statistics, like average or standard deviation




In [ ]:
# Average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(len(accuracy_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')